In [28]:
from minio import Minio
from minio.error import *
import boto3
from io import BytesIO

from minio.commonconfig import REPLACE, CopySource

import logging
import time 

host = 'localhost:9000'
access_key = 'minio'
secret_key = 'miniostorage'

client = Minio(host, access_key=access_key,
                    secret_key=secret_key, secure=False)

In [29]:
#Configurações de log
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    
    filename='/home/acsantos/Documents/Facens_Architecture-for-Data-Processing/logs/minio/move_files_' + time.strftime('%Y%m%d-%H%M%S') +'.log',
    level=logging.DEBUG,
    datefmt='%Y%m%d-%H%M%S'
)


In [32]:
def fn_move_files(bucket=None, sourcePath=None, destinationPath=None):
    
    objects = client.list_objects(bucket, prefix=sourcePath)
        
    for obj in objects:
        
        objectName = obj.object_name.replace(sourcePath, '')
        objectName = destinationPath + objectName
        
        result = client.copy_object(
            bucket,
            objectName,
            CopySource(bucket, obj.object_name),
        )
        
        client.remove_object(bucket, obj.object_name)

In [33]:
fn_move_files(bucket='bronze', sourcePath='topics/steam/partition=0/', destinationPath='processed_files/steam/')